In [1]:
import argparse
from webcam_utils import realtime_emotions
from prediction_utils import prediction_path
from emotion_recommend import recommend_by_emotion
import threading
import logging
import time

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 


Using TensorFlow backend.


In [2]:
def __get_logger():
    __logger = logging.getLogger('logger')
    formatter = logging.Formatter(
        '%(levelname)s##%(asctime)s##%(message)s')
    fileHandler = logging.FileHandler('./log.txt')
    fileHandler.setFormatter(formatter)
    __logger.addHandler(fileHandler)
    __logger.setLevel(logging.DEBUG)

    return __logger

logger = __get_logger()

In [3]:
# TODO: Socekt list (Need socket num to distinguish)

class SocketList():
    def __init__(self):
        self.lock = threading.Lock()
        self.socket_list = None
    
    def is_empty(self):
        if self.socket_list is None:
            return True
        else:
            return False

    def send(self, data):
        self.lock.acquire()
        try:
            self.socket_list.send(data)
        finally:
            self.lock.release()
        
    def add_socket(self, socket):
        self.lock.acquire()
        try:
            self.socket_list = socket
        finally:
            self.lock.release()

In [4]:
# for running realtime emotion detection
import easydict

def emotion_detection(img_str):
    return prediction_path(img_str)
    

In [5]:
from flask import Flask, request, jsonify

def send_to_socket(data):
    global socket_list
    if not socket_list.is_empty():
        logger.info('write to socket %s', data)
        try:
            socket_list.send(data.encode("utf-8"))
        except Exception as e:
            logger.error(e)
    else:
        logger.info('No socket')

def rest_api_work():
    app = Flask(__name__)
    host = 'localhost'
    port = 5000

    @app.route('/emotion', methods = ['POST'])
    def analysisEmotion():
        logger.info('Emotion Request')
        try:
            imageData = request.get_json()['image']
            emotion_request_json = jsonify({'result': emotion_detection(imageData)})
            send_to_socket(emotion_request_json.get_data(as_text=True))
            return emotion_request_json
        except Exception as e:
            logger.error(e)

    @app.route('/recommendations', methods = ['POST'])
    def analysisEmotionAndReturnRecommends():
        logger.info('Recommendations Request')
        try:
            imageData = request.get_json()['image']
            path = request.get_json()['path']
            emotion_request_json = jsonify({'result': 'test'})
            send_to_socket(emotion_request_json.get_data(as_text=True))
            return emotion_request_json
        except Exception as e:
            logger.error(e)

    @app.route('/wakeup', methods = ['GET'])
    def wakeup():
        logger.info('Wakeup Request')
        try:
            wakeup_request_json = jsonify({'status': 'open'}) 
            send_to_socket(wakeup_request_json.get_data(as_text=True))
            return wakeup_request_json
        except Exception as e:
            logger.error(e)

    app.run(host=host, port=port)

In [6]:
import socketserver
import socket

class MyTcpHandler(socketserver.BaseRequestHandler):
    def handle(self):
        try:
            logger.info('New socket added') 
            global socket_list
            socket_list.add_socket(self.request)

            while True:
                time.sleep(1)
            return
        except Exception as e:
            logger.error(e)

In [7]:
def socket_work():
    HOST = 'localhost'
    PORT = 6000

    def runServer():
        try:
            logger.info('Server started')
            server = socketserver.TCPServer((HOST,PORT),MyTcpHandler)
            server.serve_forever()
        except KeyboardInterrupt:
            logger.info('Server ended')
    
    runServer()

In [8]:
global socket_list
socket_list = SocketList()

th1 = threading.Thread(target=rest_api_work)
th2 = threading.Thread(target=socket_work)

th1.start()
th2.start()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
